## Neural Network Experiment
- The test set given are after a specific time, we have an assumption: if we limit the train to after that date, what will happen?

In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from keras import metrics

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

2021-12-29 20:26:41.313536: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-29 20:26:41.313585: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Get the data as Numpy arrays
train_set = pd.read_csv('../data/subtrain/train_1.tsv', sep='\t')
train_set['cross_city'] = train_set['cross_city'].astype('int')
train_set['cross_state'] = train_set['cross_state'].astype('int')
valid_set = pd.read_csv('../data/subtrain/valid_1.tsv', sep='\t')
valid_set['cross_city'] = valid_set['cross_city'].astype('int')
valid_set['cross_state'] = valid_set['cross_state'].astype('int')

In [3]:
train_set[:5]
train_set.shape
valid_set.shape

,record_number,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,category_id,item_price,quantity,bt,package_size,weight,tz_dis,dis,cross_city,cross_state,acc_hour,pay_hour,acc_date,target
0,1503096,2,0.0,2,9,10,21.00,1,1,0,0.0,0.0,1.778966,1,1,7,19,20190104,3
1,1503097,0,3.5,3,5,3,1.40,1,0,0,0.0,-1.0,0.823542,1,1,21,9,20190107,2
2,1503098,7,0.0,2,5,4,0.99,1,0,0,13.0,-1.0,1.184160,1,1,15,8,20190109,2
3,1503099,5,6.9,2,5,0,9.00,1,1,0,0.0,-4.0,3.897320,1,1,14,15,20190128,2
4,1503100,0,3.5,3,5,6,28.60,1,0,0,3.0,-1.0,1.003335,1,1,14,13,20190102,3


(5889444, 19)

(654382, 19)

In [51]:
x_train_valid = pd.concat([train_set, valid_set])

In [52]:
c1 = pd.get_dummies(x_train_valid.shipment_method_id, prefix='sm')
c2 = pd.get_dummies(x_train_valid.category_id, prefix='ci')
c3 = pd.get_dummies(x_train_valid.package_size, prefix='ps')
c4 = pd.get_dummies(x_train_valid.cross_city, prefix='cc')
c5 = pd.get_dummies(x_train_valid.cross_state, prefix='cs')
x_train_valid = pd.concat([x_train_valid.drop(['record_number', 'shipment_method_id', 'category_id', 'package_size', 'cross_city', 'cross_state'], axis=1), 
               c1, c2, c3, c4, c5], axis=1)

In [53]:
x_train = x_train_valid[:train_set.shape[0]]
x_valid = x_train_valid[train_set.shape[0]:]

In [54]:
hlabel = x_train[(x_train['target'] == 2) | (x_train['target'] == 3)]
hlabel.shape
llabel = x_train[(x_train['target'] != 2) & (x_train['target'] != 3)]
llabel.shape
new_x_train = (pd.concat([llabel, hlabel.sample(frac=0.5, axis=0)], axis=0)).sample(frac=1, axis=0)
new_x_train

(4327644, 83)

(1561800, 83)

,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_price,quantity,bt,weight,tz_dis,dis,acc_hour,...,ps_3,ps_4,ps_5,ps_6,cc_-1,cc_0,cc_1,cs_-1,cs_0,cs_1
1248598,9.00,2,9,34.00,1,1,0.0,-4.0,3.524621,5,...,0,0,0,0,0,0,1,0,0,1
3258461,3.99,3,5,29.00,1,1,8.0,-3.0,2.972921,11,...,0,0,0,0,0,0,1,0,0,1
5045626,4.81,3,5,79.90,1,0,8.0,2.0,3.193460,18,...,0,0,0,0,0,0,1,0,0,1
422998,0.00,2,8,9.98,1,1,4.0,-2.0,1.061526,12,...,0,0,0,0,0,0,1,0,0,1
4733800,0.00,3,5,11.79,1,0,0.0,-1.0,0.915957,12,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3095451,0.00,3,5,26.49,1,0,8.0,-4.0,2.865003,16,...,0,0,0,0,0,0,1,0,0,1
4187079,0.00,3,5,10.50,1,0,2.0,1.0,2.910743,14,...,0,0,0,0,0,0,1,0,0,1
3558345,0.00,2,8,16.95,1,0,5.0,2.0,3.790801,15,...,0,0,0,0,0,0,1,0,0,1
4272569,0.00,2,5,27.99,1,0,0.0,1.0,2.699353,17,...,0,0,0,0,0,0,1,0,0,1


In [55]:
y_train = new_x_train.target
y_valid = x_valid.target
x_train = new_x_train.drop(['target'], axis=1)
x_valid = x_valid.drop(['target'], axis=1)

print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

(3725622, 82) (3725622,) (654382, 82) (654382,)


In [56]:
# Build a simple model
inputs = keras.Input(shape=(82,))
x = layers.Dense(512, activation="sigmoid")(inputs)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation="sigmoid")(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation="tanh")(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation="tanh")(x)
x = layers.Dropout(0.4)(x)
outputs = layers.Dense(1, activation="relu")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 82)]              0         
                                                                 
 dense_10 (Dense)            (None, 512)               42496     
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 256)               131328    
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 256)               65792     
                                                                 
 dropout_10 (Dropout)        (None, 256)               0   

In [57]:
def custom_asymmetric_eval(y_train, preds):
    nd_preds = tf.math.round(preds)
    residual = y_train - nd_preds
    loss = tf.where(residual < 0, residual * -0.6, residual * 0.4) 
    return tf.math.reduce_mean(loss)

In [58]:
callback = tf.keras.callbacks.EarlyStopping(monitor='custom_asymmetric_eval', patience=3)
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error", metrics = [custom_asymmetric_eval])

batch_size = 32
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=3, validation_data=(x_valid, y_valid),callbacks=[callback])

2021-12-29 20:53:42.047917: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2444008032 exceeds 10% of free system memory.


Epoch 1/3
116426/116426 [==============================] - 549s 5ms/step - loss: 3.3609 - custom_asymmetric_eval: 0.6168 - val_loss: 2.5247 - val_custom_asymmetric_eval: 0.4852
Epoch 2/3
116426/116426 [==============================] - 545s 5ms/step - loss: 3.3471 - custom_asymmetric_eval: 0.6122 - val_loss: 2.3451 - val_custom_asymmetric_eval: 0.4852
Epoch 3/3
116426/116426 [==============================] - 542s 5ms/step - loss: 3.3446 - custom_asymmetric_eval: 0.6111 - val_loss: 2.3471 - val_custom_asymmetric_eval: 0.4852


## Conclusion

Well, the performance do increases for NN, but still too high compared to tree models.